In [1]:
from autoencoder import Autoencoder
import torch.nn.functional as F
import torch


In [ ]:
def reconstruction_loss(x, x_hat):
    return F.mse_loss(x_hat, x, reduction='sum')

def kl_divergence_loss(self, mu, stddev):
    return 0.5 * torch.sum(torch.exp(stddev) + mu**2 - 1.0 - stddev)

def loss_function(self, x, x_hat, mu, stddev):
    return self.reconstruction_loss(x, x_hat) + self.kl_divergence_loss(mu, stddev)

In [ ]:
def train(model, device, optimizer, train_loader, epoch, log_interval):
    model.to(device)
    model.train()
    